#### Bibliotecas

In [34]:
import requests
import json
import pandas as pd
import datetime

#### Extração dos Dados

API: Portal da Transparência do TCE SP (Tribunal de Contas do Estado de São Paulo)

Dados de despesas dos munícipios: Santos, Praia Grande, São Vicente, Guaruja, Caraguatatuba

In [2]:
#API municipios
url_municipios = 'https://transparencia.tce.sp.gov.br/api/json/municipios'

In [3]:
ret_municipio = requests.get(url_municipios)
municipio = json.loads(ret_municipio.content)


In [4]:
#Cria uma lista com todos os municípios que contêm dados de despesas disponíveis 
municipio_json = municipio
df_municipios = pd.DataFrame(municipio_json)
lista_municipios = df_municipios['municipio'].values.tolist()
mun_lista = lista_municipios


In [5]:
#Função para validar se os municípios escolhidos estão na lista do TCE
def valida_municipio(municipio=list, lista=list):
    for i in range(0, 5):
        if lista.index(municipio[i]):
            try:
                print(f"Município {municipio[i]} contêm dados de despesas disponíveis")
            except ValueError:
                print("O município não contém na lista")


In [6]:
municipios = ['santos', 'praia-grande', 'sao-vicente', 'guaruja', 'caraguatatuba']


In [7]:
municipios_validados = valida_municipio(municipios, mun_lista)


Município santos contêm dados de despesas disponíveis
Município praia-grande contêm dados de despesas disponíveis
Município sao-vicente contêm dados de despesas disponíveis
Município guaruja contêm dados de despesas disponíveis
Município caraguatatuba contêm dados de despesas disponíveis


In [8]:
#Função para criar dataframe pandas para cada município, consolidando todos os anos de 2014 a 2019
def df_despesa_municipio(municipio):

    url_despesas = 'https://transparencia.tce.sp.gov.br/api/json/despesas'
    municipio = municipio
    ano = [2014, 2015, 2016, 2017, 2018, 2019]
    mes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    lista_ano = list()
    df_despesas_unico = pd.DataFrame()


    for i_ano in ano:

        for i_mes in mes:
            retorno_api = f"{url_despesas}/{municipio}/{i_ano}/{i_mes}"
            retorno_despesas = requests.get(retorno_api)
            arquivo_despesas = json.loads(retorno_despesas.content)
            df_despesas_mes = pd.DataFrame(arquivo_despesas)
            df_despesas_unico = df_despesas_unico.append(df_despesas_mes, ignore_index=True)

        lista_ano.append(i_ano)

    print(f'Arquivo de Despesas criado com sucesso! \n Município: {municipio} \n Ano: {lista_ano} \n Mês: {mes}')

    return df_despesas_unico


### Transformação dos dados

In [18]:
def deleta_duplicidade(df_source):

    df = df_source.drop_duplicates()

    return df


In [11]:
def deleta_colunas(df_source):

    df = df_source.drop(columns=['evento', 'nr_empenho', 'id_fornecedor'])
    
    return df

In [12]:
def deleta_linha_vazia(df_source):

    df = df_source.dropna(how='all')

    return df

In [13]:
def rename_uppercase(df_source):

    lista = ['orgao', 'nm_fornecedor']

    for col in lista:
        df_source[col] = df_source[col].str.title()

    return df_source


In [14]:
def renomemar_coluna(df_source):

   df_source = df_source.rename(columns={'nm_fornecedor': 'nome_fornecedor', 'vl_despesa': 'valor_despesa'})

   return df_source   

In [87]:
def cria_nova_tabela_municipio(df_source, valor):

    df_source = df_source.assign(municipio=valor)

    df_source['dt_emissao_despesa'] = pd.to_datetime(
        df_source['dt_emissao_despesa'], format='%d/%m/%Y')
    
    df_source['ano'] = df_source['dt_emissao_despesa'].dt.year
    
    df_source = df_source.drop(
        columns=['orgao', 'nome_fornecedor', 'dt_emissao_despesa']
    )

    df_source = df_source.groupby(['municipio', 'ano', 'mes'])[
        'valor_despesa'].count()
    
    df_source = df_source.to_frame()
    
    df_source['valor_despesa'] = df_source['valor_despesa'].astype(float)


    return df_source


In [96]:
def cria_nova_tabela_orgaos(df_source):



    df_source['dt_emissao_despesa'] = pd.to_datetime(
        df_source['dt_emissao_despesa'], format='%d/%m/%Y')

    df_source['ano'] = df_source['dt_emissao_despesa'].dt.year

    df_source = df_source.drop(
        columns=['nome_fornecedor', 'dt_emissao_despesa'])

    df_source = df_source.groupby(['orgao', 'ano'])['valor_despesa'].count()

    df_source = df_source.to_frame()

    df_source['valor_despesa'] = df_source['valor_despesa'].astype(float)



    return df_source

In [ ]:
# Juntando dataframes
df_despesa_municipio = pd.concat([tabela_1, tabela_3])

#### Municipio de Praia Grande

In [ ]:
#Cria data frame
df_praia_grande = df_despesa_municipio('praia-grande')

In [19]:
#Chama as funções de transformação
df_mun_praia_grande = deleta_duplicidade(df_praia_grande)
df_mun_praia_grande = deleta_colunas(df_mun_praia_grande)
df_mun_praia_grande = deleta_linha_vazia(df_mun_praia_grande)
df_mun_praia_grande = rename_uppercase(df_mun_praia_grande)
df_mun_praia_grande = renomemar_coluna(df_mun_praia_grande)


In [88]:
#Cria novo dataframe visão despesa municipio por ano e mes
df_despesa_mun_praia_grande = cria_nova_tabela_municipio(df_mun_praia_grande, 'praia grande')


In [111]:
df_despesa_mun_praia_grande


valor_despesa
municipio    ano  mes                     
praia grande 2014 Abril             7065.0
                  Agosto            7658.0
                  Dezembro          9991.0
                  Fevereiro         5280.0
                  Janeiro           3671.0
...                                    ...
             2019 Maio              7896.0
                  Março             5805.0
                  Novembro          8782.0
                  Outubro           9177.0
                  Setembro          7805.0

[72 rows x 1 columns]

In [97]:
#Cria novo dataframe visão despesa orgao por ano 
df_despesa_orgaos_praia_grande = cria_nova_tabela_orgaos(df_mun_praia_grande)


In [110]:
df_despesa_orgaos_praia_grande


valor_despesa
orgao                                              ano                
Câmara Municipal De Praia Grande                   2014         2264.0
                                                   2015         2275.0
                                                   2016         2244.0
                                                   2017         2347.0
                                                   2018         2143.0
                                                   2019         2035.0
Instituto De Previdência Municipal De Praia Grande 2014         1600.0
                                                   2015         1534.0
                                                   2016         1593.0
                                                   2017         1454.0
                                                   2018         1633.0
                                                   2019         1550.0
Prefeitura Municipal De Praia Grande               2014        81619.0
                                                   2015        77713.0
                                                   2016        77560.0
                                                   2017        79514.0
                                                   2018        83286.0
                                                   2019        87688.0

#### Município Santos

In [ ]:
df_santos = df_despesa_municipio('santos')

In [100]:
#Chama as funções de transformação
df_mun_df_santos = deleta_duplicidade(df_santos)
df_mun_df_santos = deleta_colunas(df_mun_df_santos)
df_mun_df_santos = deleta_linha_vazia(df_mun_df_santos)
df_mun_df_santos = rename_uppercase(df_mun_df_santos)
df_mun_df_santos = renomemar_coluna(df_mun_df_santos)


In [101]:
#Cria novo dataframe visão despesa municipio por ano e mes
df_despesa_mun_santos = cria_nova_tabela_municipio(
    df_mun_df_santos, 'santos')


In [112]:
df_despesa_mun_santos


valor_despesa
municipio ano  mes                     
santos    2014 Abril             9712.0
               Agosto           10323.0
               Dezembro         12790.0
               Fevereiro         8577.0
               Janeiro           8143.0
...                                 ...
          2019 Maio              8378.0
               Março             8194.0
               Novembro         10023.0
               Outubro           9363.0
               Setembro          8645.0

[72 rows x 1 columns]

In [102]:
#Cria novo dataframe visão despesa orgao por ano
df_despesa_orgaos_santos = cria_nova_tabela_orgaos(df_mun_df_santos)


In [113]:
df_despesa_orgaos_santos


valor_despesa
orgao                                              ano                
Caixa De Assistência Ao Servidor Público Munici... 2014         2256.0
                                                   2015         2050.0
                                                   2016        10306.0
                                                   2017         9274.0
                                                   2018         9067.0
                                                   2019         9464.0
Câmara Municipal De Santos                         2014         2195.0
                                                   2015         2690.0
                                                   2016         2393.0
                                                   2017         2465.0
                                                   2018         2174.0
                                                   2019         2127.0
Fundação Arquivo E Memória De Santos               2014         2339.0
                                                   2015         1829.0
                                                   2016         1351.0
                                                   2017         1354.0
                                                   2018         1199.0
                                                   2019         1101.0
Fundação Parque Tecnológico De Santos              2014          855.0
                                                   2015          811.0
                                                   2016          697.0
                                                   2017          840.0
                                                   2018          656.0
                                                   2019          546.0
Fundação Pró-Esporte De Santos                     2014         1075.0
                                                   2015          902.0
                                                   2016          721.0
                                                   2017          788.0
                                                   2018          914.0
                                                   2019          822.0
Instituto De Previdência Social Dos Servidores ... 2014         2731.0
                                                   2015         2610.0
                                                   2016         2347.0
                                                   2017         1882.0
                                                   2018         2134.0
                                                   2019         3215.0
Prefeitura Municipal De Santos                     2014       107804.0
                                                   2015       102179.0
                                                   2016       109925.0
                                                   2017        88548.0
                                                   2018        80717.0
                                                   2019        85775.0

#### Município São Vicente

In [ ]:
df_sao_vicente = df_despesa_municipio('sao-vicente')


In [105]:
#Chama as funções de transformação
df_mun_df_sao_vicente = deleta_duplicidade(df_sao_vicente)
df_mun_df_sao_vicente = deleta_colunas(df_mun_df_sao_vicente)
df_mun_df_sao_vicente = deleta_linha_vazia(df_mun_df_sao_vicente)
df_mun_df_sao_vicente = rename_uppercase(df_mun_df_sao_vicente)
df_mun_df_sao_vicente = renomemar_coluna(df_mun_df_sao_vicente)


In [106]:
#Cria novo dataframe visão despesa municipio por ano e mes
df_despesa_mun_sao_vicente = cria_nova_tabela_municipio(
    df_mun_df_sao_vicente, 'sao vicente')


In [114]:
df_despesa_mun_sao_vicente


valor_despesa
municipio   ano  mes                     
sao vicente 2014 Abril             7218.0
                 Agosto            6876.0
                 Dezembro         10627.0
                 Fevereiro         5570.0
                 Janeiro           5968.0
...                                   ...
            2019 Maio              5488.0
                 Março             4936.0
                 Novembro          5700.0
                 Outubro           5883.0
                 Setembro          5316.0

[72 rows x 1 columns]

In [107]:
#Cria novo dataframe visão despesa orgao por ano
df_despesa_orgaos_sao_vicente = cria_nova_tabela_orgaos(df_mun_df_sao_vicente)


In [115]:
df_despesa_orgaos_sao_vicente


valor_despesa
orgao                                     ano                
Caixa De Saúde E Pec.Serv.Mun.São Vicente 2014         6866.0
                                          2015         6363.0
                                          2016         4368.0
                                          2017         5464.0
                                          2018         5680.0
                                          2019         5273.0
Câmara Municipal De São Vicente           2014         1061.0
                                          2015         1114.0
                                          2016         1134.0
                                          2017         1297.0
                                          2018         1414.0
                                          2019         1402.0
Instituto Prev.Serv.Mun.São Vicente       2014         2560.0
                                          2015         2287.0
                                          2016         2348.0
                                          2017         2577.0
                                          2018         2616.0
                                          2019         2661.0
Prefeitura Municipal De São Vicente       2014        54385.0
                                          2015        42776.0
                                          2016        51478.0
                                          2017        51951.0
                                          2018        53090.0
                                          2019        54375.0
Serviço De Saúde De São Vicente           2014        17506.0
                                          2015        11857.0
                                          2016          104.0

#### Município Guarujá

In [ ]:
df_guaruja = df_despesa_municipio('guaruja')

In [ ]:
#Chama as funções de transformação
df_mun_df_guaruja = deleta_duplicidade(df_guaruja)
df_mun_df_guaruja = deleta_colunas(df_mun_df_guaruja)
df_mun_df_guaruja = deleta_linha_vazia(df_mun_df_guaruja)
df_mun_df_guaruja = rename_uppercase(df_mun_df_guaruja)
df_mun_df_guaruja = renomemar_coluna(df_mun_df_guaruja)


In [ ]:
#Cria novo dataframe visão despesa municipio por ano e mes
df_despesa_mun_guaruja = cria_nova_tabela_municipio(
    df_mun_df_guaruja, 'guaruja')


In [ ]:
#Cria novo dataframe visão despesa orgao por ano
df_despesa_orgaos_guaruja = cria_nova_tabela_orgaos(df_mun_df_guaruja)


#### Caraguatatuba

In [ ]:
df_caraguatatuba = df_despesa_municipio('caraguatatuba')


In [ ]:
#Chama as funções de transformação
df_mun_caraguatatuba = deleta_duplicidade(df_caraguatatuba)
df_mun_caraguatatuba = deleta_colunas(df_mun_caraguatatuba)
df_mun_caraguatatuba = deleta_linha_vazia(df_mun_caraguatatuba)
df_mun_caraguatatuba = rename_uppercase(df_mun_caraguatatuba)
df_mun_caraguatatuba = renomemar_coluna(df_mun_caraguatatuba)


In [ ]:
#Cria novo dataframe visão despesa municipio por ano e mes
df_despesa_mun_caraguatatuba = cria_nova_tabela_municipio(
    df_mun_caraguatatuba, 'caraguatatuba')


In [ ]:
#Cria novo dataframe visão despesa orgao por ano
df_despesa_orgaos_caraguatatuba = cria_nova_tabela_orgaos(df_mun_caraguatatuba)


Juntando os dataframes

In [ ]:
tabela_despesa_municipio = df_despesa_mun_praia_grande.append(
    [df_despesa_mun_santos, df_despesa_mun_sao_vicente, df_despesa_mun_guaruja, df_despesa_mun_caraguatatuba])


In [109]:
tabela_despesa_municipio


valor_despesa
municipio    ano  mes                     
praia grande 2014 Abril             7065.0
                  Agosto            7658.0
                  Dezembro          9991.0
                  Fevereiro         5280.0
                  Janeiro           3671.0
...                                    ...
sao vicente  2019 Maio              5488.0
                  Março             4936.0
                  Novembro          5700.0
                  Outubro           5883.0
                  Setembro          5316.0

[216 rows x 1 columns]